In [ ]:
#this whole thing took multiple hours to run
!pip install transformers
#might not need torch, but it was complaining that I didn't have Tensor or PyTorch
#so i just did both
!pip install torch
!pip install TensorFlow
!pip install datasets

from transformers import CLIPProcessor
from datasets import load_dataset, DownloadConfig


dl_config = DownloadConfig(resume_download=True,
        num_proc=8,
        force_download=True)

# Load 'general' instance of dataset
dataset = load_dataset('dali-does/clevr-math', download_config=dl_config)

# Load version with only multihop in test data
dataset_multihop = load_dataset('dali-does/clevr-math', 'multihop',
                                download_config=dl_config)


model_path = "openai/clip-vit-base-patch32"
extractor = CLIPProcessor.from_pretrained(model_path)
def transform_tokenize(e):
  e['image'] = [image.convert('RGB') for image in e['image']]
  return extractor(text=e['question'],
                           images=e['image'],
                           padding=True)
# no argument called padding, so I removed? seems like this is taken care of in transform_tokenize
#this part takes the most time
dataset = dataset.map(transform_tokenize,
                      batched=True,
                      num_proc=8)

dataset_subtraction = dataset.filter(lambda e:
        e['template'].startswith('subtraction'), num_proc=4)

In [ ]:
# code from the website
'''
from transformers import CLIPPreprocessor
from datasets import load_dataset, DownloadConfig


dl_config = DownloadConfig(resume_download=True,
        num_proc=8,
        force_download=True)

# Load 'general' instance of dataset
dataset = load_dataset('dali-does/clevr-math', download_config=dl_config)

# Load version with only multihop in test data
dataset_multihop = load_dataset('dali-does/clevr-math', 'multihop',
                                download_config=dl_config)


model_path = "openai/clip-vit-base-patch32"
extractor = CLIPProcessor.from_pretrained(model_path)
def transform_tokenize(e):
  e['image'] = [image.convert('RGB') for image in e['image']]
  return extractor(text=e['question'],
                           images=e['image'],
                           padding=True)

dataset = dataset.map(transform_tokenize,
                      batched=True,
                      num_proc=8,
                      padding='max_length')

dataset_subtraction = dataset.filter(lambda e:
        e['template'].startswith('subtraction'), num_proc=4)
'''

In [ ]:
dataset